In [1]:
#Imports
from google.colab import drive
import pandas as pd
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense, LSTM, Flatten, Embedding, Dropout , Activation, GRU, Flatten, Input, Bidirectional, GlobalMaxPool1D, Convolution1D, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/My Drive/Fixy/merged_mi_final_X.csv')
print(data.shape)
data.head()

(9507636, 2)


,line,y
0,Arkadaşıyla tepelerde slalom yapıyormuş.,1
1,Sarhoş gibi görünüyor muydu?,0
2,Neden Hel X karşısınız?,1
3,Sizin için yapabileceğimiz başka bir şey var mı?,0
4,Eğer Raymond Becker bütçe X çekerse...,1


In [4]:
X = data['line']
y = data['y']

In [5]:
vocab_size = 15000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X)
list_tokenized_train = tokenizer.texts_to_sequences(X)

In [6]:
maxlen = 7
X_train_padded = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y, test_size=0.2, stratify=y)

In [8]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True, verbose=1)

In [9]:
embed_size = 128
model = Sequential()
model.add(Embedding(vocab_size, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1920000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,060,097
Trainable params: 2,060,097
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])

batch_size = 10000
epochs = 3
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping_cb])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6845497 samples, validate on 760611 samples
Epoch 1/6
6845497/6845497 [==============================] - 70s 10us/step - loss: 0.1624 - auc: 0.9517 - accuracy: 0.9332 - val_loss: 0.1176 - val_auc: 0.9843 - val_accuracy: 0.9530
Epoch 2/6
6845497/6845497 [==============================] - 69s 10us/step - loss: 0.1148 - auc: 0.9869 - accuracy: 0.9538 - val_loss: 0.1146 - val_auc: 0.9884 - val_accuracy: 0.9540
Epoch 3/6
3050000/6845497 [============>.................] - ETA: 36s - loss: 0.1113 - auc: 0.9888 - accuracy: 0.9549

KeyboardInterrupt: ignored

In [16]:
pred = tokenizer.texts_to_sequences(["sonumuz böyle x olacaktı"])
print(pred)
maxlen = 7
padded_pred = pad_sequences(pred, maxlen=maxlen)
print(padded_pred)
model.predict(padded_pred)

#sonumuz böyle x olacaktı
#gelme X istiyorlar
#seninle gelsem X bilemedim
#gelişi X bekliyorlardı

############ben sizinle gel X

[[19, 640, 216, 1]]
[[  0   0   0  19 640 216   1]]


array([[0.9990182]], dtype=float32)

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

y_pred = model.predict_classes(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data: %.2f%%" % (accuracy * 100.0))

print(confusion_matrix(y_test, y_pred ))
print(classification_report(y_test, y_pred, digits=4))
print(roc_auc_score(y_test, y_pred))

Accuracy on Test Data: 95.41%
[[910361  40403]
 [ 46972 903792]]
              precision    recall  f1-score   support

           0     0.9509    0.9575    0.9542    950764
           1     0.9572    0.9506    0.9539    950764

    accuracy                         0.9541   1901528
   macro avg     0.9541    0.9541    0.9540   1901528
weighted avg     0.9541    0.9541    0.9540   1901528

0.9540501112789295


In [18]:
model.save("Model_mi.h5")
print("Saved model to disk")

Saved model to disk


In [19]:
model.load_weights("/content/Model_mi.h5")

In [20]:
with open('tokenizer_mi.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)